# I - Playing with objects on Mangaki: Work, Artist, Staff, Rating

Mangaki Work objects belong to three categories: anime, manga or album.

In [1]:
from collections import Counter

Counter(Work.objects.values_list('category__slug', flat=True))

Counter({'anime': 11027, 'manga': 4503, 'album': 1})

## Get work from ID

In [2]:
death_note = Work.objects.get(id=1)
death_note

<Work: Death Note>

In [3]:
for staff in death_note.staff_set.all():
    print(staff)

Tetsuro Araki, réalisateur de Death Note
Yoshihisa Hirano, compositeur de Death Note
Tsugumi Ôba, auteur de Death Note


## Find a work by title

In [4]:
for work in Work.objects.filter(title__contains='adoka').order_by('id'):
    print('#{} ({}): {} ({} ratings)'.format(work.id, work.category.slug, work.title, work.rating_set.count()))

#33 (anime): Mahou Shoujo Madoka★Magica (728 ratings)
#176 (anime): Puella Magi Madoka Magica the Movie Part 3: Rebellion (433 ratings)
#179 (anime): Puella Magi Madoka Magica the Movie Part 1: Beginnings (315 ratings)
#180 (anime): Puella Magi Madoka Magica the Movie Part 2: Eternal (279 ratings)
#181 (anime): Mahou Shoujo Madoka★Magica Movie 3: Hangyaku no Monogatari - Magica Quartet x Nisioisin (45 ratings)
#6971 (manga): Puella Magi Madoka Magica (90 ratings)
#6973 (manga): Puella Magi Madoka★Magica: The different story (82 ratings)
#14090 (anime): Puella Magi Madoka Magica Concept Movie (5 ratings)
#20241 (anime): Puella Magi Madoka Magica Side Story: Magia Record (1 ratings)


## Find an artist by name

In [5]:
shinbou = Artist.objects.get(name__contains='Shinbou')
shinbou

<Artist: Akiyuki Shinbou>

By the way, what works did Shinbou make?

In [6]:
for staff in shinbou.staff_set.all():
    print(staff)

Akiyuki Shinbou, réalisateur de Tsukuyomi: Moon Phase
Akiyuki Shinbou, réalisateur de Nisemonogatari
Akiyuki Shinbou, réalisateur de Nisekoi
Akiyuki Shinbou, réalisateur de Sasami-san@Ganbaranai
Akiyuki Shinbou, réalisateur de Hidamari Sketch
Akiyuki Shinbou, réalisateur de Pani Poni Dash!
Akiyuki Shinbou, réalisateur de Magical Girl Lyrical Nanoha
Akiyuki Shinbou, réalisateur de Denpa Onna to Seishun Otoko Special
Akiyuki Shinbou, réalisateur de Nekomonogatari Black: Tsubasa Family
Akiyuki Shinbou, réalisateur de ef ~ A Tale of Memories
Akiyuki Shinbou, réalisateur de Sayonara Zetsubou Sensei
Akiyuki Shinbou, réalisateur de Mahou Shoujo Madoka★Magica
Akiyuki Shinbou, réalisateur de Le Portrait de Petit Cossette
Akiyuki Shinbou, réalisateur de Arakawa Under the Bridge
Akiyuki Shinbou, réalisateur de Bakemonogatari


Is there an artist who had different roles in anime/manga?

In [7]:
from collections import defaultdict

roles = defaultdict(set)
for staff in Staff.objects.select_related('artist', 'role'):
    roles[staff.artist_id, staff.artist].add(staff.role.slug)

In [8]:
artists = list(roles.keys())
artists.sort(key=lambda artist: len(roles[artist]), reverse=True)  # Ordered by the number of different taken roles

Let's display the top 10:

In [9]:
for artist in artists[:10]:
    print(artist, roles[artist])

(209, <Artist: Hayao Miyazaki>) {'writer', 'director', 'mangaka', 'author'}
(795, <Artist: Mamoru Hosoda>) {'writer', 'director', 'mangaka', 'author'}
(794, <Artist: Makoto Shinkai>) {'writer', 'director', 'mangaka', 'author'}
(1148, <Artist: Satoshi Kon>) {'writer', 'director', 'mangaka', 'author'}
(100, <Artist: Katsuhiro Otomo>) {'writer', 'director', 'mangaka', 'author'}
(2042, <Artist:  >) {'director', 'author', 'composer'}
(927, <Artist: Keiji Gotoh>) {'writer', 'director', 'mangaka'}
(509, <Artist: Gorō Taniguchi>) {'writer', 'director', 'mangaka'}
(881, <Artist: Keiichi Suzuki>) {'writer', 'mangaka', 'composer'}
(1503, <Artist: Eiichirô Oda>) {'writer', 'mangaka', 'author'}


If you replace the `set` above by a `list` (and replace `add` with `append`), you will find out that **Osamu Tezuka** is the most productive artist of all time (unsurprisingly).

Is there an artist that was both a writer and a composer? (Like Shunji Iwai.)

In [10]:
for artist in artists:
    if 'writer' in roles[artist] and 'composer' in roles[artist]:
        print(artist)

(881, <Artist: Keiichi Suzuki>)


What did he do?

In [11]:
for staff in Artist.objects.get(id=881).staff_set.all():
    print(staff)

Keiichi Suzuki, compositeur de Tokyo Godfathers
Keiichi Suzuki, mangaka de Flic à  Tokyo
Keiichi Suzuki, scénariste de Flic à  Tokyo


Actually, these are different authors that share the same name ^^'

## Browse ratings for an anime

In [12]:
madoka = Work.objects.get(id=33)
print('There are {} ratings for {}.'.format(madoka.rating_set.count(), madoka.title))

There are 728 ratings for Mahou Shoujo Madoka★Magica.


Let's display the first 5 ones:

In [13]:
madoka_ratings = madoka.rating_set.values_list('choice', flat=True)
print(madoka_ratings[:5])  # Display first 5 ratings

<QuerySet ['like', 'neutral', 'like', 'like', 'like']>


Count them:

In [14]:
from collections import Counter
Counter(madoka_ratings)

Counter({'like': 325,
         'neutral': 76,
         'dislike': 47,
         'willsee': 94,
         'wontsee': 144,
         'favorite': 42})

## A few stats

### How many users?

In [15]:
User.objects.count()

2429

## How many work objects?

In [16]:
Work.objects.count()

15531

## How many ratings?

In [17]:
Rating.objects.count()

359319

So our rating matrix is filled less than 1%.

## How many works that received at least one rating?

In [18]:
len(set(Rating.objects.values_list('work_id', flat=True)))

11090

## How many favorites?

In [19]:
favorites = Rating.objects.filter(choice='favorite')
favorites.count()

15340

## How many works added to favorites at least once?

In [20]:
len(set(favorites.values_list('work_id', flat=True)))

2421